In [205]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

In [206]:
pd.reset_option('all')
pd.set_option('display.max_rows', 100)

In [207]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [208]:
# Pick occupation
def pick_occupation(occupation):
    if occupation == 'travelAgents':
        GPT_input_occupation = 'travel agents'
        plot_title_occupation = 'Travel Agents'
        occupation_code = '41-3041'
    elif occupation == 'insuranceUnderwriters':
        GPT_input_occupation = 'insurance underwriters'
        plot_title_occupation = 'Insurance Underwriters'
        occupation_code = '13-2053'
    
    occupation_folder = f'{data_path}/daily_tasks_occupations_analysis/{occupation}'
    return GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder


In [209]:
# set alpha as AI quality metric
n = 1000
epsilon = 1e-8
alpha_list = np.linspace(epsilon, 1-epsilon, n).tolist()

In [210]:
# Pick occupation and initialize variables
occupation = 'travelAgents'
# occupation = 'insuranceUnderwriters'

GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

### Initialize input-output paths

In [211]:
#suffix = 'MAX_' # for when partition cost used MAX of machine costs in partition (somewhat like a least common multiple)
suffix = ''

In [212]:
# Manual DAG
input_path = f'{occupation_folder}/{occupation}_manual_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_{suffix}manual.csv'

# # First Last Task DAG
# input_path = f'{occupation_folder}/v1/{occupation}_firstLastTaskGPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_{suffix}firstLastTask.csv'

# # Conditioned First Last Task DAG
# input_path = f'{occupation_folder}/v1/{occupation}_conditionedGPT_fromFirstLastTask_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_{suffix}firstLastTask_conditioned.csv'

# # Partitioned DAG
# input_path = f'{occupation_folder}/v1/{occupation}_partitionedGPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_{suffix}partitioned.csv'

# # Conditioned Partitioned DAG
# input_path = f'{occupation_folder}/v1/{occupation}_conditionedGPT_fromPartitioned_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_{suffix}partitioned_conditioned.csv'

In [213]:
# read DAG
dag_df = pd.read_csv(input_path)

# remove edges if comment column labeled with "TriangleRemovedFlag" (edge is there for plotting purposes and is not part of the actual DAG)
if 'comment' in dag_df.columns:
    dag_df = dag_df[~dag_df['comment'].str.endswith('TriangleRemovedFlag')]

# extract list of tasks and create a dictionary for indexing tasks
tasks_list = list(set(dag_df['source']).union(set(dag_df['target'])))
tasks_dict = {i: node for i, node in enumerate(tasks_list, start=0)}

# create numpy array of adjacency matrix
adjacency_matrix = np.zeros((len(tasks_list), len(tasks_list)), dtype=int)

# Populate the adjacency matrix
aux_dict = {value: key for key, value in tasks_dict.items()}
for _, row in dag_df.iterrows():
    source_index = aux_dict[row['source']]
    target_index = aux_dict[row['target']]
    adjacency_matrix[source_index, target_index] = 1

tasks_dict

{0: 'Authorize reinsurance of policy when risk is high.',
 1: 'Decline excessive risks.',
 2: 'Examine documents to determine degree of risk from factors such as applicant health, financial standing and value, and condition of property.',
 3: 'Evaluate possibility of losses due to catastrophe or excessive insurance.',
 4: 'Write to field representatives, medical personnel, or others to obtain further information, quote rates, or explain company underwriting policies.',
 5: 'Review company records to determine amount of insurance in force on single risk or group of closely related risks.',
 6: 'Decrease value of policy when risk is substandard and specify applicable endorsements or apply rating to ensure safe, profitable distribution of risks, using reference materials.'}

In [214]:
# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')

# Define a break-even difficulty for base AI quality (alpha)
# Above break-even difficulty threshold task is done manually
# As AI quality (alpha) goes up break-even difficulty goes up
for index, alpha in enumerate(alpha_list):
    if index % np.floor(n/4) == np.floor(n/4) - 1:
        pretty_label = str(np.round(alpha,2)*100).split('.')[0]
        #tasks_stats[f'be_difficulty_{pretty_label}'] = np.log(tasks_stats['machine_cost'] / tasks_stats['human_cost']) / np.log(alpha)

# add task_dict key and reset index
aux_dict = {value: key for key, value in tasks_dict.items()}
tasks_stats['dict_index'] = tasks_stats.apply(lambda row: aux_dict[row.task], axis=1)
tasks_stats = tasks_stats.sort_values(by='dict_index')
tasks_stats = tasks_stats.set_index('dict_index', drop=False)
tasks_stats.index.name = None

tasks_stats.iloc[:,:-1]

,task,human_cost,machine_cost,difficulty
0,Authorize reinsurance of policy when risk is h...,60,10,0.85
1,Decline excessive risks.,60,5,0.75
2,Examine documents to determine degree of risk ...,60,30,0.85
3,Evaluate possibility of losses due to catastro...,240,30,0.85
4,"Write to field representatives, medical person...",30,30,0.75
5,Review company records to determine amount of ...,120,30,0.75
6,Decrease value of policy when risk is substand...,60,10,0.85


In [215]:
# create dictionaries for human cost, machine cost, and difficulty
M_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['human_cost']))
A_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['machine_cost']))
D_dict = dict(zip(tasks_stats['dict_index'], tasks_stats['difficulty']))

# print stats
tasks_stats.iloc[:,1:-1].sum()


human_cost      630.00
machine_cost    145.00
difficulty        5.65
dtype: float64

### Generate all possible partition schemes for the set of tasks (ignoring structre of the DAG)

In [216]:
from itertools import combinations

def partitions(set_):
    if not set_:
        yield []
        return
    for i in range(1, len(set_) + 1):
        for part in combinations(set_, i):
            remaining = set(set_) - set(part)
            if not remaining:
                yield [list(part)]
            else:
                for b in partitions(list(remaining)):
                    yield [list(part)] + b

def generate_unique_partitions(numbers):
    all_partitions = set()
    for partition in partitions(numbers):
        # Create a frozenset of frozensets to make each partition hashable and order-independent
        partition_set = frozenset(frozenset(part) for part in partition)
        all_partitions.add(partition_set)
    
    # Convert the frozensets back to lists for the final output
    unique_partitions = [list(map(list, partition)) for partition in all_partitions]

    # Sort elements
    unique_partitions = sorted([sorted(x) for x in unique_partitions], key=len)
    return unique_partitions

### Check if partition scheme is "valid" (i.e., if its non-singleton partitions are a connected graph)

In [217]:
def is_connected(matrix):
    # Number of nodes in the matrix
    num_nodes = matrix.shape[0]
    
    # Visited array to keep track of visited nodes
    visited = np.zeros(num_nodes, dtype=bool)
    
    # Helper function to perform DFS
    def dfs(node):
        visited[node] = True
        # Visit all the neighbors of the current node
        for neighbor in range(num_nodes):
            if matrix[node, neighbor] == 1 and not visited[neighbor]:
                dfs(neighbor)
            elif matrix[neighbor, node] == 1 and not visited[neighbor]:
                dfs(neighbor)
    
    # Start DFS from the first node (node 0)
    dfs(0)
    
    # If all nodes are visited, the matrix is connected
    return np.all(visited)


def validate_partition_using_connectedness(adjacency_matrix, tasks_list):
    # Return valid if Singleton
    if len(tasks_list) == 1:
        return True
    # Check if partition forms connected graph
    else:
        # Subset original adjacency matrix
        subset_matrix = adjacency_matrix[np.ix_(tasks_list, tasks_list)]

        # check if subset matrix is a connected graph
        subset_matrix_connected = is_connected(subset_matrix)

        # return true if connected and false otherwise
        return subset_matrix_connected

In [218]:
# Generate list of numbers for tasks in occupation
tasks_list_numbers = list(range(len(tasks_list)))

# Generate all possible partitioning schemes
all_partitions = generate_unique_partitions(tasks_list_numbers)

# Get valid partitioning schemes
valid_partitions = []
for partition_scheme in all_partitions:

    # Set valid partitions count to 0
    valid_partition_count = 0
    for partition in partition_scheme:
        valid_partition = validate_partition_using_connectedness(adjacency_matrix, partition)
        if valid_partition:
            valid_partition_count += 1
    
    # If number of valid partitions within a partition scheme is equal to 
    # number of partitions in partition scheme then partition scheme is valid
    if valid_partition_count == len(partition_scheme):
        valid_partitions.append(partition_scheme)

# Print stats
print(f'Number of all possible partitioning schemes: {len(all_partitions)}')
print(f'Number of valid partitioning schemes given DAG structure: {len(valid_partitions)}')

# print some partitions
print('\nExample partitions:')
for partition in valid_partitions[10:20]:
    print(partition)

Number of all possible partitioning schemes: 877
Number of valid partitioning schemes given DAG structure: 170

Example partitions:
[[0], [1, 2, 3, 4, 5, 6]]
[[0, 1, 2, 5, 6], [3, 4]]
[[0, 2, 3, 4, 6], [1, 5]]
[[0, 1, 2, 3, 5, 6], [4]]
[[0, 6], [1, 5], [2, 3, 4]]
[[0], [1, 2, 3, 4, 6], [5]]
[[0, 3, 4, 6], [1], [2, 5]]
[[0, 3, 4], [1, 2, 6], [5]]
[[0], [1, 2, 4, 5, 6], [3]]
[[0, 1, 2, 4, 6], [3], [5]]


### Compute minimum cost for a given partition

In [219]:
def get_partition_boundary(adjacency_matrix, partition):
    # create a matrix whose columns are nodes not in the partition and whose rows are nodes in the partition
    # (subset adjacency matrix to outgoing edges of partition nodes --i.e., rows-- and incoming edges of non-partition nodes --i.e., columns.)
    reduced_matrix = np.delete(adjacency_matrix, partition, axis=1) 
    reduced_matrix = reduced_matrix[partition, :]

    # find nodes in partition w/ an edge to non-partition nodes
    partition_boundary_tasks = [i for i in partition if np.any(reduced_matrix[partition.index(i), :])]

    return partition_boundary_tasks


def compute_partition_cost(adjacency_matrix, M_dict, A_dict, D_dict, AI_quality, partition):
    # initialize partition done manually as False 
    # (only if partition is singleton and manual cost <= automated cost partition is done manually)
    partition_done_manually = False
    
    # calculate automation cost of doing partition
    # first, get partition boundary tasks if partition contains more than one task
    if len(partition) > 1:
        partition_boundary_tasks = get_partition_boundary(adjacency_matrix, partition)
        
        
    # if partition boundary has zero length partition is invalid
        if len(partition_boundary_tasks) == 0:
            partition_cost = 100000000 # (value doesn't matter)
            partition_done_manually = False
            partition_is_valid = False
            return partition_cost, partition_done_manually, partition_is_valid


    # if partition is a singleton pick minimum of manual and machine cost
    if len(partition) == 1:
        partition_is_valid = True

        # calculate manual cost
        manual_cost = sum(M_dict[key] for key in partition)

        # calculate machine cost
        AI_cost = sum(A_dict[key] for key in partition)
        difficulty = sum(D_dict[key] for key in partition)
        automation_cost = AI_cost * (AI_quality ** (-1 * difficulty))
        
        # pick the minimum of the two
        if manual_cost < automation_cost:
            partition_cost = manual_cost
            partition_done_manually = True 
        else:
            partition_cost = automation_cost
    

    # if partition not a singleton calculate automation cost and return if partition passes a sanity check
    if len(partition) > 1:

        # calculate manual cost
        manual_cost = sum(M_dict[key] for key in partition)


        # calculate machine cost
        # first get boundary tasks in partition
        partition_boundary_tasks = get_partition_boundary(adjacency_matrix, partition)

        # if partition has no boundary tasks partition is invalid
        if len(partition_boundary_tasks) == 0:
            partition_cost = 100000000 # (value doesn't matter)
            partition_is_valid = False
            return partition_cost, partition_done_manually, partition_is_valid 
        
        # if partition has at least one boundary task calculate automation cost using boundary tasks for calculating machine costs and partition tasks for difficulty
        if len(partition_boundary_tasks) > 0:
            AI_cost = sum(A_dict[key] for key in partition_boundary_tasks)
            difficulty = sum(D_dict[key] for key in partition)
            automation_cost = AI_cost * (AI_quality ** (-1 * difficulty))

        # sanity check partition validity: if manual cost < automation cost partition is invalid (should not have been formed)
        if manual_cost < automation_cost:
            partition_cost = 100000000 # (value doesn't matter)
            partition_is_valid = False
        else:
            partition_cost = automation_cost
            partition_is_valid = True
    
    return partition_cost, partition_done_manually, partition_is_valid


### Compute costs of all "valid" execution plans
#### New check for validity: automated cost of tasks in non-singleton partition must be less than human costs doing partition tasks separately

In [220]:
########## Random Thought: maybe better to sort valid_partitions on descending partition order to avoid recalculating single node partitions everytime? 
# tho the downside is that we have to first do the heavy calculations first...


def execute_plans(adjacency_matrix, valid_partitions, M_dict, A_dict, D_dict, alpha):
    execution_plan = []
    execution_plan_manual_tasks = []
    execution_cost = []
    counter = 0
    for partition_scheme in valid_partitions:
        # initialize partition scheme cost
        # and partitions that are done manually
        partition_scheme_cost = 0
        manual_partitions = []
        
        for partition in partition_scheme:
            # calculate partition cost 
            partition_cost, partition_done_manually, partition_is_valid = compute_partition_cost(adjacency_matrix, M_dict, A_dict, D_dict, alpha, partition)
        
            # if (automated) partition is invalid ignore partition scheme
            # and stop calculating costs of further partitions
            if not partition_is_valid:
                break

            if partition_done_manually:
                manual_partitions.append(partition)

            # if (automated) partition passes sanity check
            # add this partition's cost to partition scheme cost
            partition_scheme_cost += partition_cost
        
        # if stopped because an (automated) partition wasn't valid
        # ignore current partition scheme and continue
        if not partition_is_valid:
            continue
        
        # if partition scheme makes sense append costs
        execution_plan.append(partition_scheme)
        execution_plan_manual_tasks.append(manual_partitions)
        execution_cost.append(partition_scheme_cost)

        # if counter % (np.floor(len(valid_partitions)/3)) == 0:
        #     print(partition_scheme)
        #     print(partition_scheme_cost)
        #     print('\n')
        # counter += 1

    return execution_plan, execution_plan_manual_tasks, execution_cost


random.seed(1)
execution_plan, execution_plan_manual_tasks, execution_cost = execute_plans(adjacency_matrix, valid_partitions, M_dict, A_dict, D_dict, alpha)
print(f'Number of valid execution plans: {len(execution_plan)}')

# print some valid execution plans
print('\nExample Execution Plans:')
for plan in execution_plan[10:15]:
    print(plan)

Number of valid execution plans: 144

Example Execution Plans:
[[0, 2, 3, 4, 6], [1, 5]]
[[0, 1, 2, 3, 5, 6], [4]]
[[0], [1, 2, 3, 4, 6], [5]]
[[0, 3, 4, 6], [1], [2, 5]]
[[0, 3, 4], [1, 2, 6], [5]]


### Calculate minimum cost for each alpha

In [221]:
random.seed(1)

minimum_cost_list = []
number_of_optimal_schemes_list = []
optimal_execution_plan_list = []
optimal_plan_manualTasks_list = []
optimal_plan_manualTasks_count_list = []
for alpha in alpha_list:
    # get list of execution plans and costs for this alpha
    execution_plan, execution_plan_manual_tasks, execution_cost = execute_plans(adjacency_matrix, valid_partitions, M_dict, A_dict, D_dict, alpha)

    # choose minimum
    minimum_cost = min(execution_cost)
    minimum_cost_index = [index for index, value in enumerate(execution_cost) if value == minimum_cost]

    # in rare cases there are more than one optimal plan
    if len(minimum_cost_index) > 2:
        print(alpha)
        for index in minimum_cost_index:
            optimal_execution_scheme = execution_plan[index]
            optimal_execution_manual_tasks = execution_plan_manual_tasks[index]
            print(optimal_execution_scheme)
            print(optimal_execution_manual_tasks)
    else:
        optimal_execution_scheme = execution_plan[minimum_cost_index[0]]
        optimal_execution_manual_tasks = execution_plan_manual_tasks[minimum_cost_index[0]]
    
    # append lists
    minimum_cost_list.append(minimum_cost)
    number_of_optimal_schemes_list.append(len(minimum_cost_index))
    optimal_execution_plan_list.append(optimal_execution_scheme)
    optimal_plan_manualTasks_list.append(optimal_execution_manual_tasks)
    optimal_plan_manualTasks_count_list.append(len(optimal_execution_manual_tasks))

# save outputs
output_df = pd.DataFrame({
    'alpha': alpha_list,
    'optimal_schemes_count': number_of_optimal_schemes_list,
    'cost': minimum_cost_list,
    'optimal_scheme': optimal_execution_plan_list,
    'optimal_scheme_manual_tasks': optimal_plan_manualTasks_list,
    'manual_tasks_count': optimal_plan_manualTasks_count_list
})
output_df.to_csv(output_path, index=False)